# V2 Model Server (SKLearn)

In [1]:
import mlrun

> 2021-01-26 10:41:41,477 [warning] Failed resolving version info. Ignoring and using defaults
> 2021-01-26 10:41:42,879 [warning] Unable to parse server or client version. Assuming compatible: {'server_version': 'unstable', 'client_version': 'unstable'}


### Configuration and package dependencies

In [2]:
%nuclio config kind="serving"
%nuclio config spec.build.baseImage="mlrun/mlrun"

%nuclio: setting kind to 'serving'
%nuclio: setting spec.build.baseImage to 'mlrun/mlrun'


### Serving class code

In [3]:
from cloudpickle import load
from typing import List
from sklearn.datasets import load_iris
import numpy as np

import warnings 
warnings.filterwarnings('ignore')

In [4]:
class ClassifierModel(mlrun.serving.V2ModelServer):
    def load(self):
        """load and initialize the model and/or other elements"""
        model_file, extra_data = self.get_model('.pkl')
        self.model = load(open(model_file, 'rb'))

    def predict(self, body: dict) -> List:
        """Generate model predictions from sample."""
        feats = np.asarray(body['inputs'])
        result: np.ndarray = self.model.predict(feats)
        return result.tolist()

In [5]:
# nuclio: end-code

# Convert to function object

### Document and save (as a template)

In [6]:
import mlrun
fn = mlrun.code_to_function('v2-model-server', 
                            description="generic sklearn model server",
                            categories=['serving', 'ml'],
                            labels={'author': 'yaronh', 'framework': 'sklearn'},
                            code_output='.', with_doc=False)

fn.spec.default_class = 'ClassifierModel'
#print(fn.to_yaml())
fn.export()

> 2021-01-26 10:42:21,110 [info] function spec saved to path: function.yaml


### Configure and add model(s)

In [7]:
models_path = 'https://s3.wasabisys.com/iguazio/models/iris/model.pkl'
mlrun.mlconf.dbpath = mlrun.mlconf.dbpath or 'http://mlrun-api:8080'
fn.add_model('mymodel', model_path=models_path)
#fn.verbose = True

# Test models locally (using a server emulator)

In [9]:
# create an emulator (mock server) from the function configuration)
server = fn.to_mock_server()

> 2021-01-26 10:44:24,672 [info] model mymodel was loaded
> 2021-01-26 10:44:24,673 [info] Loaded ['mymodel']


### Test against the iris dataset 

In [10]:
iris = load_iris()
x = iris['data'].tolist()

In [11]:
result = server.test("/v2/models/mymodel/infer", {"inputs": x})
result.keys()

dict_keys(['id', 'model_name', 'outputs'])

##  Deploy server

In [12]:
fn.apply(mlrun.mount_v3io())

In [13]:
fn.deploy()

> 2021-01-26 10:44:54,583 [info] Starting remote function deploy
2021-01-26 10:44:54  (info) Deploying function
2021-01-26 10:44:54  (info) Building
2021-01-26 10:44:54  (info) Staging files and preparing base images
2021-01-26 10:44:54  (info) Building processor image
2021-01-26 10:45:39  (info) Build complete
2021-01-26 10:45:45  (info) Function deploy complete
> 2021-01-26 10:45:46,287 [info] function deployed, address=default-tenant.app.yh57.iguazio-cd0.com:30291


'http://default-tenant.app.yh57.iguazio-cd0.com:30291'

##  Test server

In [14]:
my_data = '''{"inputs":[[5.1, 3.5, 1.4, 0.2],[7.7, 3.8, 6.7, 2.2]]}'''
fn.invoke('/v2/models/mymodel/infer', my_data)

{'id': '3f2f7ec5-4ad6-4140-9c27-f9989af63591',
 'model_name': 'mymodel',
 'outputs': [0, 2]}